In [2]:
import os
import pandas as pd 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, VotingRegressor, AdaBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, median_absolute_error, r2_score

In [3]:
os.chdir('..')

In [4]:
test = pd.read_csv('ENG_DATA/SELECTED/12-8_test.csv', index_col = [0])
train = pd.read_csv('ENG_DATA/SELECTED/12-8_train.csv', index_col = [0])

In [5]:
x_test = test.drop(columns = 'sale_price')
y_test = test["sale_price"]
x_train = train.drop(columns = 'sale_price')
y_train = train["sale_price"]

In [6]:
cat_columns = ['full_address',
 'street_name',
 'on_market_date',
 'sale_date',
 'type',
 'neighborhood']

In [7]:
# Label encode categories
le=LabelEncoder()

# create a dataframe to store label encoded values
x_train_le = x_train.copy()
x_test_le = x_test.copy()

for i in cat_columns:
    le.fit(x_train[i])
    x_train_le[i] = le.transform(x_train[i]).copy()
    
for i in cat_columns:
    le.fit(x_test[i])
    x_test_le[i] = le.transform(x_test[i]).copy()

In [18]:
x_train_le.columns

Index(['longitude', 'latitude', 'elevation', 'zip', 'area', 'district_no',
       'subdist_no', 'cdom', 'rooms', 'baths', 'beds', 'lot_acres',
       'year_built', 'days_on_market', 'on_market_year', 'sale_year',
       'private', 'public', 'gsRating', 'enrollment', 'arson', 'assault',
       'bad checks', 'bribery', 'burglary', 'disorderly conduct',
       'driving under the influence', 'drug/narcotic', 'drunkenness',
       'embezzlement', 'extortion', 'family offenses',
       'forgery/counterfeiting', 'fraud', 'gambling', 'kidnapping',
       'larceny/theft', 'liquor laws', 'loitering', 'missing person',
       'non-criminal', 'other offenses', 'prostitution', 'recovered vehicle',
       'robbery', 'runaway', 'secondary codes', 'sex offenses, forcible',
       'sex offenses, non forcible', 'stolen property', 'suicide',
       'suspicious occ', 'trespass', 'vandalism', 'vehicle theft', 'warrants',
       'weapon laws', 'totalincidents', 'full_address', 'street_name',
       'on_mark

In [15]:
clf_f = RandomForestRegressor(n_estimators=20, max_depth=50,random_state=0)
clf_f.fit(x_train_le, y_train)
y_pred = clf_f.predict(x_test_le)

In [19]:
r2_score(y_pred, y_test)

0.6509977827036159

In [11]:
clf_ab = AdaBoostRegressor(random_state=0, n_estimators=50)
clf_ab.fit(x_train_le, y_train)
y_pred_ab = clf_ab.predict(x_test_le)

In [12]:
r2_score(y_pred_ab, y_test)

-0.6508615774444952

In [20]:
df = pd.read_csv('ENG_DATA/CLEANED/12-8_Cleaned_df.csv', index_col = [0])

In [23]:
# Merge two Dataframes on index of both the dataframes - ADD Back original list price
x_train_le = x_train_le.merge(df[["orig_list_price"]], left_index=True, right_index=True)
x_test_le = x_test_le.merge(df[["orig_list_price"]], left_index=True, right_index=True)

In [26]:
clf_f = RandomForestRegressor(n_estimators=10, max_depth=50,random_state=0)
clf_f.fit(x_train_le, y_train)
y_pred = clf_f.predict(x_test_le)

In [27]:
r2_score(y_pred, y_test)

0.9608999756014683